In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
# !pip install scikit-learn==1.5.0

In [3]:
!python -V

Python 3.10.13


In [4]:
import pickle
import pandas as pd

In [5]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [6]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [7]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
# Q1
 
import numpy as np
np.std(y_pred)

6.247488852238703

In [16]:
year = 2023
month = 3

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [18]:
# df['ride_id'] = (df.tpep_pickup_datetime.dt.year.map("{:04}".format).astype('int')/df.tpep_pickup_datetime.dt.month.map("{:02}".format).astype('int')).astype('int').astype('string')\
#       +'_' + df.index.astype('str')

In [19]:
df_result = pd.concat([df['ride_id'].reset_index(drop=True),pd.Series(y_pred,name='y_pred')], axis=1)

In [22]:
df_result.head()

,ride_id,y_pred
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486


: 

In [20]:
df_result.to_parquet(
    'df_result.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)